In [35]:
import numpy as np
import cv2

In [36]:
"""
img: The input grayscale image.
GL: Number of gray levels.
d: The distance between pixel pairs.
t: The angle (in degrees) specifying the direction of pixel pairs.
"""
def am_glcm_faster( img , GL , d , t):
    ccmm = np.zeros([GL, GL])
    if t==0:
        im_target = img[:,d:]
        im_value = img[:,:-d]
        '''
        1 2 3 4 5 6 7 8 9 10  original
        1 2 3 4 5 6 7 8 9     target
          2 3 4 5 6 7 8 9 10  value
        '''
    elif t==45:
        im_target = img[d:,d:]
        im_value = img[:-d, :-d]
    elif t==90:
        im_target = img[d:,:]
        im_value = img[:-d, :]
    elif t==135:
        im_target = img[:-d, :-d]
        im_value = img[d:,d:]
    else:
        raise NotImplementedError('t options: 0, 45, 90, 135 only')
    assert im_target.shape==im_value.shape
    ref_gls = np.unique(im_value)
    # print(ref_gls)
    for ref in ref_gls:
        #ref: 80
        target_values = im_target[im_value == ref]
        #target_values: [ 98,  89,  99, 100]
        target_gls = np.unique(target_values)
        #[ 89,  98,  99, 100]
        for target in target_gls:

               ccmm[ref, target] = (target_values==target).sum()
    return ccmm


# function to compute the image power. input could be graylevel or color.
#
def am_power(a):
    dim1 = a.shape

    if len(dim1)==2:
        sz = dim1[0] * dim1[1]
    elif len(dim1)==3:
        sz = dim1[0] * dim1[1] * dim1[2]
    pa = np.sum( a ** 2.0) / sz

    return pa

def am_entropy(nimg , N=256):
    M = nimg.shape
    ssz = M[0] * M[1]
    hist,bins = np.histogram(nimg.flatten(),N,[0,N])
    hist = hist / ssz
    ent = -np.sum( hist * np.log2(hist+0.000001))
    return ent

glcm_energy = lambda gl : np.sum(np.multiply(gl,gl))

In [37]:
test_img1 = cv2.imread('/Users/tianze/cs4243_lab/CS4243_2023_images_small/collage1.bmp', 0)
test_img2 = cv2.imread('/Users/tianze/cs4243_lab/CS4243_2023_images_small/20230324_105524gl.jpg', 0)

In [38]:
# Separate a 512*512 path of each from <0,0> to <512,512>, we call them a5, b5
a5 = test_img1[0:512, 0:512]
b5 = test_img2[0:512, 0:512]

In [39]:
glcma530 = am_glcm_faster(a5, 256, 3, 0)
glcma5390 = am_glcm_faster(a5, 256, 3, 90)
glcmb530 = am_glcm_faster(b5, 256, 3, 0)
glcmb5390 = am_glcm_faster(b5, 256, 3, 90)

ls = [['glcma530',glcma530], ['glcma5390', glcma5390], ['glcmb530', glcmb530], ['glcmb5390', glcmb5390]]
print(sorted(map(lambda x: [x[0], glcm_energy(x[1])], ls), key=lambda x: x[1]))


[['glcma5390', 23644310.0], ['glcma530', 123233206.0], ['glcmb530', 132581438.0], ['glcmb5390', 150995342.0]]


Q2

In [40]:
c = cv2.imread('//Users/tianze/cs4243_lab/CS4243_2023_images_small/IMG_0054.JPG', 0)
d = cv2.imread('//Users/tianze/cs4243_lab/CS4243_2023_images_small/6ae-007.jpg', 0)

In [41]:
L3 = np.array([1,2,1]) / 4
E3 = np.array([-1,0,1])
S3 = np.array([-1,2,-1])
llist = [L3 , E3 , S3]

laws = []
for xs in llist:
    for ys in llist:
        xs= np.reshape(xs,(3,1))
        ys = np.reshape(ys,(1,3))
        L3L3 = np.matmul(xs,ys)
        laws.append(L3L3)

ppp=[]
fff= []
for i in range(9):
    f1 = cv2.filter2D(src=c, ddepth=-1, kernel= laws[i])
    ppp.append(am_power(f1))
    fff.append(f1)

print('original image power c= ' , am_power(c) )
for i in range(9):
    print('power of c', i ,'th filter response=', ppp[i] )


ppp=[]
fff= []
for i in range(9):
    f1 = cv2.filter2D(src=d, ddepth=-1, kernel= laws[i])
    ppp.append(am_power(f1))
    fff.append(f1)

print('original image power d= ' , am_power(d) )
for i in range(9):
    print('power of d', i ,'th filter response=', ppp[i] )

original image power c=  15875.908085340141
power of c 0 th filter response= 15632.804892407947
power of c 1 th filter response= 367.35752368398494
power of c 2 th filter response= 81.87042335867245
power of c 3 th filter response= 519.2561169735623
power of c 4 th filter response= 320.66703837861354
power of c 5 th filter response= 124.87064970009018
power of c 6 th filter response= 94.2810190940469
power of c 7 th filter response= 132.59986965274604
power of c 8 th filter response= 124.51946238505309
original image power d=  7340.401925308194
power of d 0 th filter response= 7294.189411711385
power of d 1 th filter response= 45.623743654822334
power of d 2 th filter response= 16.272286983321248
power of d 3 th filter response= 102.31650108774474
power of d 4 th filter response= 69.06252356780276
power of d 5 th filter response= 38.73454133430022
power of d 6 th filter response= 40.261400471356055
power of d 7 th filter response= 74.31934372733865
power of d 8 th filter response= 56.8

Q3

In [42]:
x = cv2.imread('/Users/tianze/cs4243_lab/CS4243_2023_images_small/34.jpg', 0)
y = cv2.imread('/Users/tianze/cs4243_lab/CS4243_2023_images_small/68.JPG', 0)

Corrupt JPEG data: bad Huffman code


In [43]:
hlp = np.full((3,3), 1/9)
hhp = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])

In [44]:
px = am_power(x)
py = am_power(y)
pxlp = am_power(cv2.filter2D(src=x, ddepth=-1, kernel=hlp))
pxhp = am_power(cv2.filter2D(src=x, ddepth=-1, kernel=hhp))
pylp = am_power(cv2.filter2D(src=y, ddepth=-1, kernel=hlp))
pyhp = am_power(cv2.filter2D(src=y, ddepth=-1, kernel=hhp))

print('power of x low pass filter response=', pxlp, '\npower of x high pass filter response=', pxhp, '\npower of y low pass filter response=', pylp, '\npower of y high pass filter response=', pyhp)

power of x low pass filter response= 20040.985943554722 
power of x high pass filter response= 218.39124206679634 
power of y low pass filter response= 14851.285573111156 
power of y high pass filter response= 353.16680350349924


In [45]:
xd = cv2.resize(x, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_LINEAR)
yd = cv2.resize(y, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_LINEAR)

In [46]:
pxd = am_power(xd)
pyd = am_power(yd)
pxdlp = am_power(cv2.filter2D(src=xd, ddepth=-1, kernel=hlp))
pxdhp = am_power(cv2.filter2D(src=xd, ddepth=-1, kernel=hhp))
pydlp = am_power(cv2.filter2D(src=yd, ddepth=-1, kernel=hlp))
pydhp = am_power(cv2.filter2D(src=yd, ddepth=-1, kernel=hhp))

print('power of x low pass filter response=', pxdlp, '\npower of x high pass filter response=', pxdhp, '\npower of y low pass filter response=', pydlp, '\npower of y high pass filter response=', pydhp)

power of x low pass filter response= 20043.852923584127 
power of x high pass filter response= 750.31918285647 
power of y low pass filter response= 14803.2328976782 
power of y high pass filter response= 2755.5220748869583


In [49]:
print(pxhp/px, pylp/py, pyhp/py, pylp/py)
print(pxdhp/pxd, pxdlp/pxd, pydhp/pyd, pydlp/pyd)

0.010889743074898523 0.9973959212582383 0.023718292103678706 0.9973959212582383
0.03735868893323736 0.9979913662143187 0.18405719818066615 0.988793229418889


Q4
Specify how optical flow can be used to determine:
a. mutual velocity of an observer and an object.
b. The focus of expansion.
c. Distance of a moving object from the observer.
d. Possible collision of the object with an observer and time of collision.

Ans:
a. Mutual Velocity of an Observer and an Object:
Optical flow computes the displacement of pixels between two consecutive frames. This displacement can be used to estimate the relative velocity between the observer (camera) and an object.
The velocity vector of a point (or region) in the scene can be estimated by computing the magnitude and direction of its optical flow.
If the observer is stationary, the computed optical flow directly corresponds to the motion of the object. If both the observer and object are moving, the computed optical flow is a combination of both motions, and additional information or assumptions may be needed to separate them.

b. The Focus of Expansion:
When an observer is moving through a scene, the point in the image towards which the observer is moving is called the focus of expansion (FOE). All the optical flow vectors radiate out from this point.
By analyzing the direction of the optical flow vectors across the image, one can determine the FOE as the point where these vectors converge or originate. For instance, when driving forward on a straight road, the FOE would be somewhere in the middle of the horizon, and all the scene elements would seem to move away from that point.

c. Distance of a Moving Object from the Observer:
The magnitude of optical flow is inversely proportional to the distance of the object from the observer. Objects closer to the camera will have a larger optical flow than distant ones, given the same actual velocity.
To precisely determine the distance, calibration is needed. Once calibrated, one can use the disparity between the observed optical flow and the expected optical flow (based on object's known speed) to estimate distance.
It's important to note that this method is more qualitative than quantitative. For more precise measurements, methods like stereo vision or depth sensing are typically used in combination with optical flow.

d. Possible Collision of the Object with an Observer and Time of Collision:
If an object is on a collision course with the observer, its trajectory will lead it to the FOE. Thus, monitoring the optical flow vectors can help determine if they are converging towards the FOE.
The time to collision (TTC) can be estimated by the inverse of the rate of expansion of the optical flow from the FOE. Specifically, TTC can be approximated as the ratio of the distance of the object to its relative speed. If d is the distance of the object from the observer and v is its relative speed towards the observer (computed from optical flow), then TTC = d/v.
For more accurate calculations, it's often useful to track the object over several frames to refine the optical flow estimates and thereby refine TTC calculations.
While optical flow provides valuable insights into scene dynamics, it's essential to note that it can be affected by factors like lighting changes, occlusions, and noise. Thus, in real-world applications, it's often combined with other methods or sensors for improved robustness and accuracy.


Q5

In [50]:
fr11 = cv2.imread('/Users/tianze/cs4243_lab/Python_notebooks/set5/OF_samples/fr1.png', 0)
fr12 = cv2.imread('/Users/tianze/cs4243_lab/Python_notebooks/set5/OF_samples/fr2.png', 0)